In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
from scripts.baseline_model import MoleculeModel
from scripts.utils import generate_descriptors
from collections import defaultdict
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

In [3]:
data_train_val = pd.read_csv('../data/split/data_train_val.csv')
data_test = pd.read_csv('../data/split/data_test.csv')

In [4]:
TARGET_COLUMN = 'active'
FP_SIZE = 256
FP_RADIUS = 2

In [5]:
data_train_val = generate_descriptors(data_train_val, fp_size=FP_SIZE, fp_radius=FP_RADIUS)
data_test = generate_descriptors(data_test, fp_size=FP_SIZE, fp_radius=FP_RADIUS)

  0%|          | 0/388 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

## Final model selection

In [6]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from ITMO_FS.embedded import MOS

In [7]:
X_train_val = data_train_val.drop(columns = [TARGET_COLUMN, 'smiles'])
X_test = data_test.drop(columns = [TARGET_COLUMN, 'smiles'])
y_train_val = data_train_val.active
y_test = data_test.active

In [9]:
cols = pd.concat([X_train_val, X_test]).columns

In [10]:
scaler = MinMaxScaler()
scaler.fit(pd.concat([X_train_val, X_test]))
X_train_val = scaler.transform(X_train_val)
X_test = scaler.transform(X_test)

In [11]:
mos = MOS()

mos.fit(np.concatenate([X_train_val, X_test], axis=0), pd.concat([y_train_val, y_test]).to_numpy(), feature_names=data_train_val.columns)
X_train_val = mos.transform(X_train_val)
X_test = mos.transform(X_test)

In [33]:
X = np.concatenate([X_train_val, X_test])
y = pd.concat([y_train_val, y_test])

In [12]:
for i, col in enumerate(cols):
    if i in mos.selected_features:
        print(col)

FpDensityMorgan1
BCUT2D_MWHI
BCUT2D_MRHI
RingCount
#amine
CNS
PISA
glob
QPlogPC16
QPlogPo/w
QPlogS
CIQPlogS
QPlogKp
EA(eV)
QPlogKhsa
HumanOralAbsorption
PercentHumanOralAbsorption
#ringatoms
#in56


In [13]:
X_train_val.shape

(372, 19)

In [14]:
skf = StratifiedKFold(n_splits=4,  random_state=42, shuffle=True)

In [15]:
def get_model(model):
    '''
    Get the model from six state-of-the-art machine learning models.
    '''
    if model=='svm':
        from sklearn.svm import SVC
        names = ["Linear SVM"]
        classifiers = [
        SVC()    
        ]
    elif model=='ab':
        from sklearn.ensemble import AdaBoostClassifier
        names = ["AdaBoost"]
        classifiers = [
        AdaBoostClassifier() 
        ]
    elif model=='knn':
        from sklearn.neighbors import KNeighborsClassifier
        names = ["K-Nearest Neighbors"]
        classifiers = [
        KNeighborsClassifier()
        ]
    elif model=='rfc':
        from sklearn.ensemble import RandomForestClassifier
        names = ["Random Forest"]
        classifiers = [
        RandomForestClassifier()
        ]
    elif model=='xgboost':
        from xgboost import XGBClassifier
        names = ["XGBoost"]
        classifiers = [
        XGBClassifier()
        ]
    elif model=='mlpclassifier':
        from sklearn.neural_network import MLPClassifier
        names = ["MLPClassifier"]
        classifiers = [
        MLPClassifier()
        ]
    else:
        raise RuntimeError('Unknown classifier')
    
    return classifiers

In [16]:
parameters = {
              'svm': {'model__C': (1, 5, 10, 50, 100), 'model__probability': [True]}, 
              'ab': {'model__n_estimators': (10, 25, 50, 100, 125, 150, 200)}, 
              'knn': {'model__n_neighbors': (3, 5, 10, 50, 75, 100), 'model__leaf_size': (1, 2, 3, 5, 10, 15, 20), 
                      'model__weights': ['uniform', 'distance']}, 
              'rfc': {'model__max_depth': (2, 3, 5, 7, 10), 'model__n_estimators': (50, 100, 150, 200),
                     'model__min_samples_leaf': (1, 3, 5, 10)},
              'mlpclassifier': {'model__hidden_layer_sizes': (
                                  (25, 50, 25, 10), 
                                  (10, 15, 10),
                                  (30, 50, 20, 10)),
                                'model__alpha': (0.0001, 0.001, 0.00001, 0.01), 
                                'model__learning_rate': ['constant', 'adaptive'],
                                'model__max_iter': [500]
                               },
              'xgboost': {'model__n_estimators': (10, 25, 50, 100)}
            }

In [17]:
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

In [18]:
best_clfs_encode = {}
for name, param in parameters.items():
    model = get_model(name)[0]
    
    steps = [('model', model)]

    pipeline = Pipeline(steps=steps)
    clf = GridSearchCV(pipeline, param, scoring='roc_auc', cv=skf, n_jobs=-1)
    clf.fit(X_train_val, y_train_val)

    print(f'Best params for {name}:', clf.best_params_)
    pipe = clf.best_estimator_['model']

    print('Test ROC AUC for the best model %.2f' % roc_auc_score(y_test, pipe.predict_proba(X_test)[:,1]))
    print('Test accuracy for the best model %.2f' % accuracy_score(y_test, pipe.predict(X_test)))
    print('Test f1-score for the best model %.2f' % f1_score(y_test, pipe.predict(X_test)))
    print()
    best_clfs_encode[name] = clf.best_estimator_

Best params for svm: {'model__C': 1, 'model__probability': True}
Test ROC AUC for the best model 0.92
Test accuracy for the best model 0.85
Test f1-score for the best model 0.91

Best params for ab: {'model__n_estimators': 10}
Test ROC AUC for the best model 0.91
Test accuracy for the best model 0.85
Test f1-score for the best model 0.90

Best params for knn: {'model__leaf_size': 1, 'model__n_neighbors': 75, 'model__weights': 'distance'}
Test ROC AUC for the best model 0.95
Test accuracy for the best model 0.84
Test f1-score for the best model 0.91

Best params for rfc: {'model__max_depth': 2, 'model__min_samples_leaf': 5, 'model__n_estimators': 100}
Test ROC AUC for the best model 0.93
Test accuracy for the best model 0.87
Test f1-score for the best model 0.92



/home/lizzka239/anaconda3/envs/rdkit/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Best params for mlpclassifier: {'model__alpha': 0.0001, 'model__hidden_layer_sizes': (10, 15, 10), 'model__learning_rate': 'adaptive', 'model__max_iter': 500}
Test ROC AUC for the best model 0.93
Test accuracy for the best model 0.89
Test f1-score for the best model 0.93

[11:48:42] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best params for xgboost: {'model__n_estimators': 100}
Test ROC AUC for the best model 0.88
Test accuracy for the best model 0.81
Test f1-score for the best model 0.88



/home/lizzka239/anaconda3/envs/rdkit/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/lizzka239/anaconda3/envs/rdkit/lib/python3.9/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/lizzka239/anaconda3/envs/rdkit/lib/python3.9/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/lizzka239/anaconda3/envs/rdkit

## Best model to test

In [53]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [54]:
# model = KNeighborsClassifier(leaf_size=1, n_neighbors=75, weights='distance')
model = RandomForestClassifier(max_depth=10, min_samples_leaf=10, n_estimators=50)
model.fit(X, y)

RandomForestClassifier(max_depth=10, min_samples_leaf=10, n_estimators=50)

In [55]:
test_data = pd.read_csv('../data/preprocessed/custom_test_data.csv')

In [56]:
test_data = generate_descriptors(test_data, fp_size=FP_SIZE, fp_radius=FP_RADIUS)

  0%|          | 0/319 [00:00<?, ?it/s]

In [57]:
test = test_data.drop(columns=['smiles'])

In [58]:
test = scaler.transform(test)
test = mos.transform(test)

In [59]:
test.shape

(283, 19)

In [60]:
predicted = model.predict(test)

In [61]:
test_data['predicted'] = pd.Series(predicted) 

In [62]:
for i in list(test_data[test_data.predicted == 0].smiles):
    print(i)

CC/C(=C(\c1ccccc1)/c1ccc(cc1)OCCN(C)C)/c1ccccc1
OC[C@H]1O[C@H]([C@@H]([C@@H]1O)O)n1cnc2c1ncnc2NC1CCCCC1
OC(=O)[C@@H](c1ccc(c(c1)F)c1ccccc1)C
CCCCNC(=O)NS(=O)(=O)c1ccc(cc1)C
COCCOCCCSc1ccc(cn1)C(=O)Nc1ccc(cc1C(=O)O)C#N
OCC(=O)[C@@]1(O)[C@H](C)C[C@@H]2[C@]1(C)C[C@H](O)[C@]1([C@H]2CCC2=CC(=O)C=C[C@]12C)F
CC(=O)C[C@@H](c1c(=O)oc2c(c1O)cccc2)c1ccccc1
OCCOCn1cnc2c1nc(N)[nH]c2=O
CN(CC[C@@H](c1ccccn1)c1ccc(cc1)Cl)C
OC[C@H]1O[C@H]([C@@H]([C@@H]1O)O)n1cnc2c1ncnc2NC1CC2CC1CC2


In [64]:
test_data_csv = test_data[['smiles', 'predicted']]

In [ ]:
test_data_csv.to_csv('../data/predicted_test/custom_db_predicted.csv', inde)